In [1]:
import sys
sys.path.append('..')

import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
import time
import faiss
import xarray as xr
from geopy.distance import geodesic
import geopandas as gpd
from sklearn.preprocessing import StandardScaler
from shapely.geometry import Point
from sklearn.impute import SimpleImputer, KNNImputer
from MeteoDataset import MeteoDataset
from scipy.spatial import distance
from load_meteo_data import create_meteo_dataframe
from sklearn.neighbors import BallTree
from scipy.spatial.distance import cdist
from pathlib import Path
from constants import *
from load_era5_data import create_era5_dataframe
from eda_utils import impute_data

In [5]:
files = {'Bihar_536_Sensor_Data_Sep_2023_Screened.csv': 7, 'Bihar_536_Sensor_Data_Oct_2023_Screened.csv': 7,\
        'Bihar_536_Sensor_Data_Nov_2023_Screened.csv': 6, 'Bihar_536_Sensor_Data_Jan_2024_Screened.csv': 6,\
        'Bihar_536_Sensor_Data_Dec_2023_Screened.csv': 6, 'Bihar_512_Sensor_Data_May_Aug_Screened_Hourly.csv': 6}

nc_files = [f'{data_bihar}/era5_may_dec_2023.nc', f'{data_bihar}/era5_jan_2024.nc']

geojson_file = f'{data_bihar}/bihar.json'
pbl_file = f'{data_bihar}/PBLH_may_Dec_2023.nc'
other_params_file = f'{data_bihar}/Era5_data_May_Dec_2023.nc'
output_meteo_file = f'{data_bihar}/bihar_meteo_may_jan.pkl'
output_meteo_era5_file = f'{data_bihar}/bihar_meteo_era5_may_jan.pkl'
output_meteo_era5_imputed_file = f'{data_bihar}/bihar_meteo_era5_may_jan_iterative_imputed.pkl'
pm25_clustering_file = f'{data_bihar}/bihar_clustering_pm25.csv'

region = gpd.read_file(geojson_file)
# meteo_df = pd.read_pickle(meteo_file)

start_date, end_date = pd.Timestamp('2023-05-01 00:00:00'), pd.Timestamp('2024-02-01 00:00:00')

In [4]:
if Path(output_meteo_file).is_file():
        meteo_df = pd.read_pickle(output_meteo_file)
else:
    meteo_df = create_meteo_dataframe(files, output_meteo_file, start_date, end_date)

In [5]:
min_lon, min_lat, max_lon, max_lat = region.total_bounds
min_lon, min_lat, max_lon, max_lat

(83.320238, 24.286327, 88.29954611201047, 27.521347)

In [6]:
region.total_bounds

array([83.320238  , 24.286327  , 88.29954611, 27.521347  ])

In [7]:
if Path(output_meteo_era5_file).is_file():
        era5_df = pd.read_pickle(output_meteo_era5_file)
else:
    era5_df = create_era5_dataframe(nc_files, meteo_df, output_meteo_era5_file)

In [5]:
if Path(output_meteo_era5_imputed_file).is_file():
    imputed_df = pd.read_pickle(output_meteo_era5_imputed_file)
else:
    imputed_df = impute_data(era5_df, output_meteo_era5_imputed_file, method='iterative')

In [14]:
imputed_df.count()

timestamp    3384864
block        3384864
district     3384864
latitude     3384864
longitude    3384864
rh           3384864
temp         3384864
blh          3384864
u10          3384864
v10          3384864
kx           3384864
sp           3384864
tp           3384864
pm25         3384864
dtype: int64

In [9]:
# df_g = imputed_df.groupby(['latitude', 'longitude'])

# for loc, group in df_g:
#     print(loc, group.shape)

In [10]:
era5_df.count()

index        3384864
timestamp    3384864
block        3384864
district     3384864
latitude     3384864
longitude    3384864
rh           2040291
temp         1907280
blh          3384864
u10          3384864
v10          3384864
kx           3384864
sp           3384864
tp           3384864
pm25         2045544
dtype: int64

In [6]:
pm25_df = pd.read_csv(pm25_clustering_file)
pm25_df

,timestamp,latitude,longitude,pm25
0,2023-05-01 00:00:00,26.148000,87.457000,101.649887
1,2023-05-01 00:00:00,26.437000,85.655000,101.649887
2,2023-05-01 00:00:00,25.853724,85.295081,101.649887
3,2023-05-01 00:00:00,25.248000,84.615000,101.649887
4,2023-05-01 00:00:00,26.133000,85.655000,101.649887
...,...,...,...,...
3513595,2024-02-29 23:00:00,25.968000,87.531000,212.874730
3513596,2024-02-29 23:00:00,25.810000,87.624000,208.352062
3513597,2024-02-29 23:00:00,25.771000,87.098000,210.637114
3513598,2024-02-29 23:00:00,25.496000,85.794000,46.950000


In [7]:
df_g = pm25_df.groupby(['latitude', 'longitude'])

for loc, group in df_g:
    print(loc, group.shape)

(24.449, 84.581) (7320, 4)
(24.503, 85.017) (7320, 4)
(24.517, 84.668) (7320, 4)
(24.532, 84.916) (7320, 4)
(24.553, 83.811) (7320, 4)
(24.573, 84.803) (7320, 4)
(24.578, 85.085) (7320, 4)
(24.61, 84.147) (7320, 4)
(24.612, 84.224) (7320, 4)
(24.613, 84.669) (7320, 4)
(24.613, 86.666) (7320, 4)
(24.642, 85.495) (7320, 4)
(24.655, 84.588) (7320, 4)
(24.655, 85.397) (7320, 4)
(24.667, 84.785) (7320, 4)
(24.668, 84.44) (7320, 4)
(24.708, 85.142) (7320, 4)
(24.71476936, 84.99768066) (7320, 4)
(24.737, 85.355) (7320, 4)
(24.743, 86.728) (7320, 4)
(24.747, 86.305) (7320, 4)
(24.75, 85.556) (7320, 4)
(24.758, 84.365) (7320, 4)
(24.763, 84.982) (7320, 4)
(24.778, 86.369) (7320, 4)
(24.779, 85.687) (7320, 4)
(24.792, 85.007) (7320, 4)
(24.796, 84.078) (7320, 4)
(24.8, 85.032) (7320, 4)
(24.803, 85.232) (7320, 4)
(24.808, 84.788) (7320, 4)
(24.814, 84.846) (7320, 4)
(24.814, 87.016) (7320, 4)
(24.82, 84.642) (7320, 4)
(24.834, 86.195) (7320, 4)
(24.836, 85.414) (7320, 4)
(24.838, 85.889) (7320, 